In [1]:
import requests
import pandas as pd
import datetime

In [2]:
import xmltodict
import json

In [3]:
import re

## Daten herunterladen von API und umwandeln

In [4]:
#Abfrage für alle Vorstösse einer Partei im gewählten Zeitrahmen (1.1.14 bis 1.1.24)
r = requests.get('http://www.gemeinderat-zuerich.ch/api/geschaeft/searchdetails?q=partei all "SP" AND beginn_start > "2014-01-01 00:00:00" AND beginn_start < "2024-01-01 00:00:00" sortBy beginn_start/sort.ascending&l=de-CH')

In [5]:
#Die Arbeit mit den XML-Daten funktioniert schlecht, darum umwandeln in JSON
xml_data = r.text
data_dict = xmltodict.parse(xml_data)

## DataFrame mit Vorstössen erstellen

In [6]:
# Minidict mit den wichtigsten Angaben zu den Vorstössen. Da es nicht immer einen Mitunterzeichner gibt, braucht's einen Fail-Safe
vorstoesse = []

for hit in data_dict["SearchDetailResponse"]["Hit"]:
    titel = hit["Geschaeft"]["Titel"]
    geschaeftsart = hit["Geschaeft"]["Geschaeftsart"]
    ID = hit["Geschaeft"]["@OBJ_GUID"]
    datum = hit["Geschaeft"]["Beginn"]["Start"]["#text"]
    erstunterzeichner = hit["Geschaeft"]["Erstunterzeichner"]["KontaktGremium"]["Name"]
    partei_erst = hit["Geschaeft"]["Erstunterzeichner"]["KontaktGremium"]["Partei"]
    
    # Kontrollieren ob "Mitunterzeichner" existiert
    mitunterzeichner_data = hit["Geschaeft"].get("Mitunterzeichner")
    
    if mitunterzeichner_data:
        # Wenn es einen Mitunterzeichner gibt, dann Daten herunterladen
        mitunterzeichner = mitunterzeichner_data["KontaktGremium"]["Name"]
        partei_mit = mitunterzeichner_data["KontaktGremium"]["Partei"]
    else:
        # Wenn nicht, dann None einsetzen
        mitunterzeichner = None
        partei_mit = None
    
    status = hit["Geschaeft"]["Geschaeftsstatus"]
    
    mini_dict = {"Titel": titel,
                 "Geschäftsart": geschaeftsart,
                 "ID": ID,
                 "Datum": datum,
                 "Erstunterzeichner": erstunterzeichner,
                 "Partei Erstunterzeichner": partei_erst,
                 "Mitunterzeichner": mitunterzeichner,
                 "Partei Mitunterzeichner": partei_mit,
                 "Geschäftsstatus": status}
    
    vorstoesse.append(mini_dict)

In [7]:
df_partei = pd.DataFrame(vorstoesse)

In [8]:
df_partei

,Titel,Geschäftsart,ID,Datum,Erstunterzeichner,Partei Erstunterzeichner,Mitunterzeichner,Partei Mitunterzeichner,Geschäftsstatus
0,Stellenwert und Bedeutung der Quartierzeitunge...,Schriftliche Anfrage,59510f153e2843a7a18ba5559fb148c2,2014-01-08T00:00:00.000,Michael Kraft,SP,None,None,Abgeschlossen
1,Umsetzung der Bewilligungspflicht des «Hooliga...,Schriftliche Anfrage,10fcdfe3849243868504a94906c451a2,2014-01-15T00:00:00.000,Matthias Probst,Grüne,Florian Utz,SP,Abgeschlossen
2,Verstösse der Vertragspartnerin für die Strass...,Schriftliche Anfrage,bdfe511a27a241ebabe4a7dc69d4d8f4,2014-01-22T00:00:00.000,Walter Angst,AL,Kyriakos Papageorgiou,SP,Abgeschlossen
3,"Parkanlage Platzspitz, durchgehende Öffnung de...",Postulat,ad634817495f4993bc91ca11a025a609,2014-01-22T00:00:00.000,Linda Bär,SP,Michael Kraft,SP,Abgeschlossen
4,Personen mit unterschiedlicher Wohnsitz- und S...,Schriftliche Anfrage,dd5d3bf44263494aab83686a4bd58e7b,2014-01-29T00:00:00.000,Gabriela Rothenfluh,SP,Pawel Silberring,SP,Abgeschlossen
...,...,...,...,...,...,...,...,...,...
822,"Hagenholzstrasse, Einführung von Tempo 30 und ...",Postulat,aa6584e0ea454f80829a8fcfdfcb454b,2023-11-29T00:00:00.000,Heidi Egger,SP,Matthias Probst,Grüne,InBearbeitung
823,Einsatz von Doppelgelenk-Trolleybussen bei sch...,Schriftliche Anfrage,f80960ad45a74f1e80e5a1cf2e3c16d7,2023-12-06T00:00:00.000,Ann-Catherine Nabholz,GLP,Ivo Bieri,SP,InBearbeitung
824,Umgestaltung des Sozialinspektorats hinsichtli...,Postulat,16285bcb96c84790bb7723bd44c7c98c,2023-12-06T00:00:00.000,Luca Maggi,Grüne,Anjushka Früh,SP,InBearbeitung
825,Verstärkte Berücksichtigung von Mehrfachnutzun...,Postulat,657741aac79440cda18dc77082591f78,2023-12-06T00:00:00.000,Pascal Lamprecht,SP,Tanja Maag Sturzenegger,AL,InBearbeitung


## Resultate für Motionen und Postulate herunterladen

In [9]:
#Hiermit hole ich mir die Resultate für die Motionen und Postulate. 
#Es braucht einen doppelten Loop und Problembehebungsmassnahmen

entscheide = []

for hit in data_dict['SearchDetailResponse']['Hit']:
    # Kontrollieren, ob Ablaufschritte existiert und eine Liste ist
    ablaufschritte = hit['Geschaeft'].get('Ablaufschritte', {}).get('Aufgabe', [])
    
    if not isinstance(ablaufschritte, list):
        # Falls Aufgabe keine Liste ist, wird sie hier in eine Liste umgewandelt
        ablaufschritte = [ablaufschritte]
    
    for aufgabe in ablaufschritte:
        # Kontrollieren, ob AblaufschrittName existiert
        entscheid = aufgabe.get('AblaufschrittName')
        if entscheid is not None:
            entscheide.append({
                "HitID": hit["Geschaeft"]["@OBJ_GUID"],
                "Entscheid": entscheid,
            })

df_entscheid = pd.DataFrame(entscheide)

In [10]:
df_entscheid

,HitID,Entscheid
0,59510f153e2843a7a18ba5559fb148c2,"Eingang, Frist 3 Monate"
1,59510f153e2843a7a18ba5559fb148c2,"Stadtrat, Antwort"
2,59510f153e2843a7a18ba5559fb148c2,Kenntnisnahme
3,10fcdfe3849243868504a94906c451a2,"Eingang, Frist 3 Monate"
4,10fcdfe3849243868504a94906c451a2,"Stadtrat, Antwort"
...,...,...
3091,657741aac79440cda18dc77082591f78,"Stadtrat, Entgegennahme"
3092,657741aac79440cda18dc77082591f78,"Ablehnung, beantragt"
3093,534dfff5acf340818e6f2c352d0b0642,"Eingang, Frist 3 Monate"
3094,534dfff5acf340818e6f2c352d0b0642,"Stadtrat, Entgegennahme"


## Daten putzen

In [11]:
#Zu jedem Geschäft gibt es mehrere Verfahrensschritte. 
#Mich interessieren nur die Schlussentscheide, die ich mit den keywords heraushole

keywords = ["Überweisung", "Ablehnung", "Rückzug", "Umwandlung"]

# Ich kann ein Pattern kreieren mit meinen keyword, zusammengefügt werden sie mit dem OR operator (|)
pattern = '|'.join(map(re.escape, keywords))

# Jetzt kann ich das Pattern auf meine Entscheidspalte anwenden mit str.contains
df_entscheid_neu = df_entscheid[df_entscheid.Entscheid.str.contains(pattern, case=False, regex=True)]

In [12]:
df_entscheid_neu

,HitID,Entscheid
9,ad634817495f4993bc91ca11a025a609,"Überweisung, Frist 24 Monate"
21,1ae08ec9dfc0465ba1c5491de2584218,"Ablehnung, beantragt"
22,1ae08ec9dfc0465ba1c5491de2584218,"Überweisung, Frist 24 Monate"
31,b78c8d83f08b438ca62f7d62f6aafb2d,"Ablehnung, beantragt"
33,b78c8d83f08b438ca62f7d62f6aafb2d,"Überweisung, Frist 24 Monate"
...,...,...
3085,aa6584e0ea454f80829a8fcfdfcb454b,"Ablehnung, beantragt"
3088,16285bcb96c84790bb7723bd44c7c98c,"Stadtrat, Ablehnung"
3089,16285bcb96c84790bb7723bd44c7c98c,"Überweisung, Frist 24 Monate"
3092,657741aac79440cda18dc77082591f78,"Ablehnung, beantragt"


In [13]:
# Das Problem ist noch nicht ganz gelöst, zwei Verfahrensschritte sind geblieben, die ich nicht brauche: 
exclude_values = ["Stadtrat, Ablehnung", "Ablehnung, beantragt"]

# Für das bereinigte DataFrame behalte ich nun alle Zeilen, welche die exclude-values nicht enthalten
df_entscheid_clean = df_entscheid_neu[~df_entscheid_neu['Entscheid'].isin(exclude_values)]

In [14]:
df_entscheid_clean

,HitID,Entscheid
9,ad634817495f4993bc91ca11a025a609,"Überweisung, Frist 24 Monate"
22,1ae08ec9dfc0465ba1c5491de2584218,"Überweisung, Frist 24 Monate"
33,b78c8d83f08b438ca62f7d62f6aafb2d,"Überweisung, Frist 24 Monate"
38,94f2be0b2acd4eab8e6d7c743304b799,"Überweisung, Frist 24 Monate"
43,148ad2aca4b94e7d8530b6056a95aa23,"Überweisung, Frist 24 Monate"
...,...,...
3057,3091bf1cba9944a3b5a2d5b691440974,Rückzug
3071,f27285abae2b44fcbd0684f5db250fd4,"Überweisung, Frist 24 Monate"
3077,9efd0883dec740b3a641abd208197cbf,"Überweisung, Frist 24 Monate"
3082,2e01ac0f64924520a8223819e0af6680,"Überweisung, Frist 24 Monate"


In [15]:
#Nun sind noch verschiedene Varianten für Überweisung verblieben, die ich noch vereinheitliche und in neuer Spalte speichere
def clean(value):
    if 'Überweisung' in value:
        return 'überweisung'
    
    return value

#df_entscheid_clean["_entscheidbereinigt"] = df_entscheid_clean["Entscheid"].apply(clean)
df_entscheid_clean['_entscheidbereinigt'] = df_entscheid_clean['Entscheid']
df_entscheid_clean.loc[df_entscheid_clean['_entscheidbereinigt'].str.contains("Überweisung"), '_entscheidbereinigt'] = "Überweisung"
df_entscheid_clean

C:\Users\hudecj\AppData\Local\Temp\ipykernel_81268\3655237080.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_entscheid_clean['_entscheidbereinigt'] = df_entscheid_clean['Entscheid']


,HitID,Entscheid,_entscheidbereinigt
9,ad634817495f4993bc91ca11a025a609,"Überweisung, Frist 24 Monate",Überweisung
22,1ae08ec9dfc0465ba1c5491de2584218,"Überweisung, Frist 24 Monate",Überweisung
33,b78c8d83f08b438ca62f7d62f6aafb2d,"Überweisung, Frist 24 Monate",Überweisung
38,94f2be0b2acd4eab8e6d7c743304b799,"Überweisung, Frist 24 Monate",Überweisung
43,148ad2aca4b94e7d8530b6056a95aa23,"Überweisung, Frist 24 Monate",Überweisung
...,...,...,...
3057,3091bf1cba9944a3b5a2d5b691440974,Rückzug,Rückzug
3071,f27285abae2b44fcbd0684f5db250fd4,"Überweisung, Frist 24 Monate",Überweisung
3077,9efd0883dec740b3a641abd208197cbf,"Überweisung, Frist 24 Monate",Überweisung
3082,2e01ac0f64924520a8223819e0af6680,"Überweisung, Frist 24 Monate",Überweisung


In [16]:
#Hier führe ich die beiden Tabellen zusammen anhand der ID resp. HitID

df_neu = df_partei.copy()

df_partei_final = df_neu.merge(df_entscheid_clean, how='left', left_on='ID', right_on='HitID')[["Titel", "Geschäftsart", "ID", "Datum", "Erstunterzeichner", "Partei Erstunterzeichner", "Mitunterzeichner", "Partei Mitunterzeichner", "Geschäftsstatus", "Entscheid", '_entscheidbereinigt']]
df_partei_final

,Titel,Geschäftsart,ID,Datum,Erstunterzeichner,Partei Erstunterzeichner,Mitunterzeichner,Partei Mitunterzeichner,Geschäftsstatus,Entscheid,_entscheidbereinigt
0,Stellenwert und Bedeutung der Quartierzeitunge...,Schriftliche Anfrage,59510f153e2843a7a18ba5559fb148c2,2014-01-08T00:00:00.000,Michael Kraft,SP,None,None,Abgeschlossen,NaN,NaN
1,Umsetzung der Bewilligungspflicht des «Hooliga...,Schriftliche Anfrage,10fcdfe3849243868504a94906c451a2,2014-01-15T00:00:00.000,Matthias Probst,Grüne,Florian Utz,SP,Abgeschlossen,NaN,NaN
2,Verstösse der Vertragspartnerin für die Strass...,Schriftliche Anfrage,bdfe511a27a241ebabe4a7dc69d4d8f4,2014-01-22T00:00:00.000,Walter Angst,AL,Kyriakos Papageorgiou,SP,Abgeschlossen,NaN,NaN
3,"Parkanlage Platzspitz, durchgehende Öffnung de...",Postulat,ad634817495f4993bc91ca11a025a609,2014-01-22T00:00:00.000,Linda Bär,SP,Michael Kraft,SP,Abgeschlossen,"Überweisung, Frist 24 Monate",Überweisung
4,Personen mit unterschiedlicher Wohnsitz- und S...,Schriftliche Anfrage,dd5d3bf44263494aab83686a4bd58e7b,2014-01-29T00:00:00.000,Gabriela Rothenfluh,SP,Pawel Silberring,SP,Abgeschlossen,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
827,"Hagenholzstrasse, Einführung von Tempo 30 und ...",Postulat,aa6584e0ea454f80829a8fcfdfcb454b,2023-11-29T00:00:00.000,Heidi Egger,SP,Matthias Probst,Grüne,InBearbeitung,NaN,NaN
828,Einsatz von Doppelgelenk-Trolleybussen bei sch...,Schriftliche Anfrage,f80960ad45a74f1e80e5a1cf2e3c16d7,2023-12-06T00:00:00.000,Ann-Catherine Nabholz,GLP,Ivo Bieri,SP,InBearbeitung,NaN,NaN
829,Umgestaltung des Sozialinspektorats hinsichtli...,Postulat,16285bcb96c84790bb7723bd44c7c98c,2023-12-06T00:00:00.000,Luca Maggi,Grüne,Anjushka Früh,SP,InBearbeitung,"Überweisung, Frist 24 Monate",Überweisung
830,Verstärkte Berücksichtigung von Mehrfachnutzun...,Postulat,657741aac79440cda18dc77082591f78,2023-12-06T00:00:00.000,Pascal Lamprecht,SP,Tanja Maag Sturzenegger,AL,InBearbeitung,NaN,NaN


In [17]:
#Diese Resultate speichern
df_partei_final.to_csv('SP_Vorstoesse_14_23_mit_Resultat.csv', index=False)

## Das Problem mit den umgewandelten Postulaten lösen

In [18]:
#Ein Problem bleibt noch: Bei Motionen, die in Postulate umgewandelt wurden, ist das Ergebnis nicht verzeichnet. 
#Weil es wenige Fälle sind, schaue ich das von Hand in den dazugehörigen PDF nach.
df_partei_final[df_partei_final["_entscheidbereinigt"] == "Umwandlung in Postulat"]

,Titel,Geschäftsart,ID,Datum,Erstunterzeichner,Partei Erstunterzeichner,Mitunterzeichner,Partei Mitunterzeichner,Geschäftsstatus,Entscheid,_entscheidbereinigt
45,Neuregelung des Anspruchs auf einen bezahlten ...,Motion,a1e77423a3d84860966c118473988c84,2015-01-21T00:00:00.000,Min Li Marti,SP,Jean-Daniel Strub,SP,Abgeschlossen,Umwandlung in Postulat,Umwandlung in Postulat
130,Bereitstellung einer Liegenschaft für den Betr...,Motion,b09f96c7f7ea42b5822e8771736b0640,2016-04-20T00:00:00.000,Christine Seidler,SP,None,None,Abgeschlossen,Umwandlung in Postulat,Umwandlung in Postulat
171,Pilotprojekt für eine vermehrte Berücksichtigu...,Motion,c9c859637737445ebdb563eefd9675ab,2016-11-16T00:00:00.000,Katharina Prelicz-Huber,Grüne,Gabriela Rothenfluh,SP,Abgeschlossen,Umwandlung in Postulat,Umwandlung in Postulat
202,Bereitstellung von mehr Wasserfläche für Freiz...,Motion,d4d610a5c6394a33bc67a0a45b39b1e2,2017-05-17T00:00:00.000,Marianne Aubert,SP,Marco Denoth,SP,Abgeschlossen,Umwandlung in Postulat,Umwandlung in Postulat
221,Rahmenkredit zur Stärkung der Partizipation de...,Motion,63a194ccf8754e29a51f44fa8e3f53d5,2017-09-13T00:00:00.000,Christine Seidler,SP,Jean-Daniel Strub,SP,Abgeschlossen,Umwandlung in Postulat,Umwandlung in Postulat
233,Neue Nutzung für den bisherigen Schiessstand H...,Motion,b3b7b404584d4a0b9232f4e9f1a3bd5f,2017-12-06T00:00:00.000,Pascal Lamprecht,SP,Markus Baumann,GLP,Abgeschlossen,Umwandlung in Postulat,Umwandlung in Postulat
251,Rahmenkredit für Massnahmen zum Schutz der Bev...,Motion,1dcb2d04a7b749d0a1d6c51a3feb4d41,2018-03-21T00:00:00.000,Markus Knauss,Grüne,Simone Brander,SP,InBearbeitung,Umwandlung in Postulat,Umwandlung in Postulat
377,Erhöhung des Grundbedarfs in der Sozialhilfe,Motion,af11035faa584fb88bc2e969b2ecd657,2019-10-23T00:00:00.000,Katharina Prelicz-Huber,Grüne,Vera Ziswiler,SP,Abgeschlossen,Umwandlung in Postulat,Umwandlung in Postulat
414,Bedarfsgerechte Erneuerung und Modernisierung ...,Motion,87ed0567b90e409baf2950f8a850df88,2020-01-29T00:00:00.000,Pascal Lamprecht,SP,Marcel Bührig,Grüne,Abgeschlossen,Umwandlung in Postulat,Umwandlung in Postulat
425,"Bahnhof Hardbrücke, Umgestaltung zu einem voll...",Motion,3db8dc22337d49b18a7ce2bbf32e6a01,2020-03-04T00:00:00.000,Urs Helfenstein,SP,Ernst Danner,EVP,Abgeschlossen,Umwandlung in Postulat,Umwandlung in Postulat


In [19]:
#Um schneller zum Ziel zu kommen, lasse ich mir die urls zu den Geschäften automatisch erstellen
df_umwandlung = df_partei_final[df_partei_final["_entscheidbereinigt"] == "Umwandlung in Postulat"]

In [20]:
url_liste = []
base = "https://www.gemeinderat-zuerich.ch/geschaefte/detail.php?gid="

for elem in df_umwandlung["ID"]:
    url_liste.append(base + elem)
    

In [21]:
url_liste

['https://www.gemeinderat-zuerich.ch/geschaefte/detail.php?gid=a1e77423a3d84860966c118473988c84',
 'https://www.gemeinderat-zuerich.ch/geschaefte/detail.php?gid=b09f96c7f7ea42b5822e8771736b0640',
 'https://www.gemeinderat-zuerich.ch/geschaefte/detail.php?gid=c9c859637737445ebdb563eefd9675ab',
 'https://www.gemeinderat-zuerich.ch/geschaefte/detail.php?gid=d4d610a5c6394a33bc67a0a45b39b1e2',
 'https://www.gemeinderat-zuerich.ch/geschaefte/detail.php?gid=63a194ccf8754e29a51f44fa8e3f53d5',
 'https://www.gemeinderat-zuerich.ch/geschaefte/detail.php?gid=b3b7b404584d4a0b9232f4e9f1a3bd5f',
 'https://www.gemeinderat-zuerich.ch/geschaefte/detail.php?gid=1dcb2d04a7b749d0a1d6c51a3feb4d41',
 'https://www.gemeinderat-zuerich.ch/geschaefte/detail.php?gid=af11035faa584fb88bc2e969b2ecd657',
 'https://www.gemeinderat-zuerich.ch/geschaefte/detail.php?gid=87ed0567b90e409baf2950f8a850df88',
 'https://www.gemeinderat-zuerich.ch/geschaefte/detail.php?gid=3db8dc22337d49b18a7ce2bbf32e6a01',
 'https://www.gemein

In [22]:
#In einem ersten Schritt passe ich nun die umgewandelten Postulate an, die abgelehnt wurden
#Mit dem loc-Befehl kann man über die Index-Nr. die richtige Zeile anwählen und dann die Spalte
df_test = df_partei_final.copy()
df_test.loc[130, "_entscheidbereinigt"] = "Ablehnung"

In [23]:
#Ein Kontrollblick
pd.set_option('display.max_rows', None)
df_test

,Titel,Geschäftsart,ID,Datum,Erstunterzeichner,Partei Erstunterzeichner,Mitunterzeichner,Partei Mitunterzeichner,Geschäftsstatus,Entscheid,_entscheidbereinigt
0,Stellenwert und Bedeutung der Quartierzeitunge...,Schriftliche Anfrage,59510f153e2843a7a18ba5559fb148c2,2014-01-08T00:00:00.000,Michael Kraft,SP,None,None,Abgeschlossen,NaN,NaN
1,Umsetzung der Bewilligungspflicht des «Hooliga...,Schriftliche Anfrage,10fcdfe3849243868504a94906c451a2,2014-01-15T00:00:00.000,Matthias Probst,Grüne,Florian Utz,SP,Abgeschlossen,NaN,NaN
2,Verstösse der Vertragspartnerin für die Strass...,Schriftliche Anfrage,bdfe511a27a241ebabe4a7dc69d4d8f4,2014-01-22T00:00:00.000,Walter Angst,AL,Kyriakos Papageorgiou,SP,Abgeschlossen,NaN,NaN
3,"Parkanlage Platzspitz, durchgehende Öffnung de...",Postulat,ad634817495f4993bc91ca11a025a609,2014-01-22T00:00:00.000,Linda Bär,SP,Michael Kraft,SP,Abgeschlossen,"Überweisung, Frist 24 Monate",Überweisung
4,Personen mit unterschiedlicher Wohnsitz- und S...,Schriftliche Anfrage,dd5d3bf44263494aab83686a4bd58e7b,2014-01-29T00:00:00.000,Gabriela Rothenfluh,SP,Pawel Silberring,SP,Abgeschlossen,NaN,NaN
5,"Juliastrasse, Einrichtung einer Begegnungszone...",Postulat,1ae08ec9dfc0465ba1c5491de2584218,2014-02-05T00:00:00.000,Jean-Daniel Strub,SP,Martin Luchsinger,GLP,Abgeschlossen,"Überweisung, Frist 24 Monate",Überweisung
6,"Teilrevision der Bau- und Zonenordnung (BZO), ...",Schriftliche Anfrage,e67c768f1c224330b68b363599ccb9be,2014-02-05T00:00:00.000,Marco Denoth,SP,Heinz F. Steger,FDP,Abgeschlossen,NaN,NaN
7,Gewinnsteuerpflicht der beiden Grossbanken UBS...,Schriftliche Anfrage,2b5a83e99e154f3abe53cda245c7e8a0,2014-02-05T00:00:00.000,Min Li Marti,SP,Florian Utz,SP,Abgeschlossen,NaN,NaN
8,Eigener Termin für die Erneuerungswahlen des S...,Postulat,b78c8d83f08b438ca62f7d62f6aafb2d,2014-02-26T00:00:00.000,Peter Küng,SP,Jean-Daniel Strub,SP,Abgeschlossen,"Überweisung, Frist 24 Monate",Überweisung
9,Durchgehende Veloverbindung aus dem Lettenquar...,Postulat,94f2be0b2acd4eab8e6d7c743304b799,2014-03-05T00:00:00.000,Simone Brander,SP,Guido Trevisan,GLP,Abgeschlossen,"Überweisung, Frist 24 Monate",Überweisung


In [24]:
pd.reset_option('display.max_rows')

In [25]:
#Nun kann ich alle übrigen umgewandelten Postulate zu "Überweisung" ändern
df_test['_entscheidbereinigt'] = df_test['_entscheidbereinigt'].replace('Umwandlung in Postulat', 'Überweisung')

In [26]:
#Diese Resultate speichern
df_test.to_csv('SP_Vorstoesse_14_23_mit_Resultat_bereinigt.csv', index=False)